In [254]:
import pandas as pd

In [363]:
df_all=pd.read_csv("df_all")
df_all

,Unnamed: 0,Job_Designation,resume_text,JD,tokens
0,0,systems administrator,web developer web developer web developer - hc...,Job Title: Web Developer\n\nJob Description:\n...,1297.0
1,1,"software developer,web developer,front end dev...",oracle database administrator oracle database ...,Job Title: Oracle Database Administrator \n\nJ...,1297.0
2,2,project manager,senior it project manager senior it project ma...,Job Title: Senior IT Project Manager\n\nLocati...,1297.0
3,3,systems administrator,lead web developer lead web developer freelanc...,Job Title: Lead Web Developer\n\nLocation: Ros...,1297.0
4,4,network administrator,driver driver driver - h beltran trucking san ...,Job Title: Security Analyst\n\nCompany: H Belt...,1297.0
...,...,...,...,...,...
1047,20551,"web developer,software developer",senior web developer & web project manager sen...,</s><s><s><s>job title: senior web developer &...,363.0
1048,20552,systems administrator,project coordinator project coordinator servic...,</s><s>job title: project coordinator/service ...,466.0
1049,20553,"web developer,software developer",sr. ui developer sr. ui developer sr. ui devel...,</s><s><s><s>job title: senior ui developer jo...,3158.0
1050,20554,"web developer,software developer",network engineer network engineer network engi...,</s><s>job title: network engineer location: t...,1637.0


In [364]:
df_all=df_all.drop(['Unnamed: 0','tokens'], axis =1)

In [365]:
df_all.duplicated().sum()

2

In [366]:
df_all= df_all.drop_duplicates()

In [367]:
df_all.isnull().sum()

Job_Designation       0
resume_text           0
JD                 1320
dtype: int64

In [368]:
df_all=df_all.dropna()

In [369]:
df_all.isnull().sum()

Job_Designation    0
resume_text        0
JD                 0
dtype: int64

In [370]:
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/hygumm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [371]:
df_all = df_all.rename(columns={'resume_text': 'Resume_Text', 'JD': 'Job_Description', 'similarity': 'Similarity','output':'Output'})

In [372]:
def lowercase(text):
    return text.lower()

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

def tokenize(text):
    return nltk.word_tokenize(text)

def lemmatize(token_list):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in token_list]

def preprocess_text(text):
    text = lowercase(text)
    text = remove_punctuation(text)
    tokens = tokenize(text)
    tokens = lemmatize(tokens)
    return ' '.join(tokens)

def preprocess_dataframe(df, columns):
    for col in columns:
        df[col] = df[col].apply(preprocess_text)
    return df

In [373]:
columns_to_preprocess = ['Resume_Text', 'Job_Description']
df_all = preprocess_dataframe(df_all, columns_to_preprocess)

In [374]:
#lemmatization and stopwords
#df_all["resume_text"] = df_all["resume_text"].apply(preprocess_text)
#df_all["Job_Designation"] = df_all["Job_Designation"].apply(preprocess_text)
#df_all["JD"] = df_all["JD"].apply(preprocess_text)

In [378]:
df_all = df_all.reset_index(drop=True)

In [379]:
Job_Description = df_all['Job_Description'].tolist()
Resume_Text = df_all['Resume_Text'].tolist()

In [381]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [383]:
vectorizer = CountVectorizer()
vectorizer.fit(df_all['Resume_Text'])

CountVectorizer()

In [384]:
similarities = []
for i in range(len(df_all)):
    resume_vector = vectorizer.transform([df_all.iloc[i]['Resume_Text']])
    job_description_vector = vectorizer.transform([df_all.iloc[i]['Job_Description']])
    similarity = cosine_similarity(resume_vector, job_description_vector)
    similarities.append(similarity[0][0])

In [385]:
df_all['Similarity'] = similarities

In [394]:
count_less_than_05 = (df_all['Similarity'] < 0.8).sum()
count_greater_than_05 = (df_all['Similarity'] > 0.8).sum()

print("Number of similarities less than 0.8:", count_less_than_05)
print("Number of similarities greater than 0.8:", count_greater_than_05)
print(count_less_than_05+count_greater_than_05)
print(df_all.shape)

Number of similarities less than 0.8: 5564
Number of similarities greater than 0.8: 16094
21658
(21658, 5)


In [387]:
df_all['Output'] = df_all['Similarity'].apply(lambda x: 'Match' if x > 0.8 else 'Not a Match')

In [388]:
df_all

,Job_Designation,Resume_Text,Job_Description,Similarity,Output
0,systems administrator,web developer web developer web developer hca ...,job title web developer job description we are...,0.860581,Match
1,"software developer,web developer,front end dev...",oracle database administrator oracle database ...,job title oracle database administrator job su...,0.855244,Match
2,project manager,senior it project manager senior it project ma...,job title senior it project manager location t...,0.844363,Match
3,systems administrator,lead web developer lead web developer freelanc...,job title lead web developer location rosevill...,0.848537,Match
4,network administrator,driver driver driver h beltran trucking san an...,job title security analyst company h beltran t...,0.840922,Match
...,...,...,...,...,...
21653,"web developer,software developer",senior web developer web project manager senio...,ssssjob title senior web developer web project...,0.856145,Match
21654,systems administrator,project coordinator project coordinator servic...,ssjob title project coordinatorservice desk an...,0.868015,Match
21655,"web developer,software developer",sr ui developer sr ui developer sr ui develope...,ssssjob title senior ui developer job descript...,0.841470,Match
21656,"web developer,software developer",network engineer network engineer network engi...,ssjob title network engineer location tempe az...,0.862504,Match


In [389]:
df_all.shape

(21658, 5)

In [390]:
df_all.to_csv('final_df_3.csv', index=False)